In [1]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
import json
import pyspark
from pyspark.sql.functions import from_json, col
from datetime import datetime
import time

In [2]:
spark = SparkSession.builder.master("local").appName("FraudCheck").getOrCreate()

22/12/08 16:41:27 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.124.1 instead (on interface eth1)
22/12/08 16:41:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/08 16:41:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("header","true").csv("DB/client.csv")

In [4]:
banned_countries = ['North Korea', 'Russia', 'Iran', 'Cuba', 'Sudan', 'Syria', 'Belgique']

In [5]:
def login_check(value):
    if value["country"] in banned_countries:
        return False
    return True

def check(row, key, expected):
    return row[key] == expected
    
def credit_card_check(value):
    
    reason = "Forbidden Country"
    if not login_check(value):
        return False, reason
        
    reason = "False Card"
    name = value["name"]
    number = value["number"]
    CVC = value["CVC"]
    expire = value["expire"]
    row = df.where(df.number==number).limit(1).collect()
    
    if len(row) == 0:
        return False, reason
    row = row[0]
    res = check(row, "number", number) and check(row, "CVC", CVC) and check(row, "expire", expire) and check(row, "name", name)
    if res:
        reason = "Card accepted"
    return res, reason
    
#credit_card_check({"name":'Stephanie Vega', "number":'180039745386698', "CVC":'187', "expire":'02/31'})

In [7]:
def send(msg, producer, topic):
    msg = json.dumps(msg)        
    producer.send(topic, msg.encode('utf-8'))
    producer.flush()
    

producer = KafkaProducer(bootstrap_servers="localhost")

consumer = KafkaConsumer('t3')

for msg in consumer:
   value = json.loads(msg.value)
   res, reason = credit_card_check(value)
   print(value)
   #time.sleep(0.1)
   part1 = {"IsLegit": str(res), "reason": reason}
   send({**value, **part1}, producer, "t4")

{'date': '2022/12/08 16:45:37', 'ID': '5c18dc5c-770f-11ed-a6ad-309c23ff59e3', 'country': 'Luxembourg', 'name': 'Nathan Atkinson', 'number': '180031332435434', 'CVC': '126', 'expire': '05/26'}
{'date': '2022/12/08 16:45:37', 'ID': '5c2b35dc-770f-11ed-a6ad-309c23ff59e3', 'country': 'Benin', 'name': 'Adam Davis', 'number': '2697911903080101', 'CVC': '844', 'expire': '12/29'}
{'date': '2022/12/08 16:45:37', 'ID': '5c3d3f7a-770f-11ed-a6ad-309c23ff59e3', 'country': 'Malta', 'name': 'Eric Hart', 'number': '2242251336342624', 'CVC': '7337', 'expire': '08/24'}
{'date': '2022/12/08 16:45:37', 'ID': '5c4ef184-770f-11ed-a6ad-309c23ff59e3', 'country': 'China', 'name': 'Joshua Santos', 'number': '3500972749338710', 'CVC': '431', 'expire': '05/26'}
{'date': '2022/12/08 16:45:37', 'ID': '5c608822-770f-11ed-a6ad-309c23ff59e3', 'country': 'Cocos Islands', 'name': 'Olivia Ross', 'number': '4927332179063655', 'CVC': '567', 'expire': '06/25'}
{'date': '2022/12/08 16:45:37', 'ID': '5c71fe36-770f-11ed-a6ad-3

{'date': '2022/12/08 16:45:43', 'ID': '5fa6f0c0-770f-11ed-a6ad-309c23ff59e3', 'country': 'Kyrgyzstan', 'name': 'Beth Hall', 'number': '374879538638961', 'CVC': '171', 'expire': '12/28'}
{'date': '2022/12/08 16:45:43', 'ID': '5fba7d70-770f-11ed-a6ad-309c23ff59e3', 'country': 'China', 'name': 'Nathan Lee', 'number': '3512178572062718', 'CVC': '466', 'expire': '02/23'}
{'date': '2022/12/08 16:45:43', 'ID': '5fcbcf30-770f-11ed-a6ad-309c23ff59e3', 'country': 'Belarus', 'name': 'Marcus Mcbride Jr.', 'number': '502028865298', 'CVC': '714', 'expire': '05/28'}
{'date': '2022/12/08 16:45:43', 'ID': '5fdd81a8-770f-11ed-a6ad-309c23ff59e3', 'country': 'Georgia', 'name': 'Joshua James', 'number': '3527361065034781', 'CVC': '484', 'expire': '05/24'}
{'date': '2022/12/08 16:45:43', 'ID': '5fef1eae-770f-11ed-a6ad-309c23ff59e3', 'country': 'Jordan', 'name': 'Lori Cole', 'number': '4809769096725967565', 'CVC': '585', 'expire': '09/31'}
{'date': '2022/12/08 16:45:43', 'ID': '600fcaa0-770f-11ed-a6ad-309c23

{'date': '2022/12/08 16:45:49', 'ID': '63718ef4-770f-11ed-a6ad-309c23ff59e3', 'country': 'Estonia', 'name': 'John Bowman', 'number': '3505463698126921', 'CVC': '124', 'expire': '10/27'}
{'date': '2022/12/08 16:45:49', 'ID': '63924298-770f-11ed-a6ad-309c23ff59e3', 'country': 'Fiji', 'name': 'Jessica Pham', 'number': '4528755795484848138', 'CVC': '588', 'expire': '01/24'}
{'date': '2022/12/08 16:45:50', 'ID': '63b30dfc-770f-11ed-a6ad-309c23ff59e3', 'country': 'Guinea-bissau', 'name': 'Brian Thompson', 'number': '2269231444819612', 'CVC': '270', 'expire': '03/27'}
{'date': '2022/12/08 16:45:50', 'ID': '63c3f1a8-770f-11ed-a6ad-309c23ff59e3', 'country': 'Afghanistan', 'name': 'Morgan Hill DDS', 'number': '180079426573610', 'CVC': '576', 'expire': '12/23'}
{'date': '2022/12/08 16:45:50', 'ID': '63d56dde-770f-11ed-a6ad-309c23ff59e3', 'country': 'Norfolk Island', 'name': 'Dr. Alexander Simmons PhD', 'number': '6582412093040079', 'CVC': '190', 'expire': '05/23'}
{'date': '2022/12/08 16:45:50', 

{'date': '2022/12/08 16:45:56', 'ID': '67c74ef8-770f-11ed-a6ad-309c23ff59e3', 'country': 'Cayman Islands', 'name': 'Sean Smith', 'number': '4778472373152865801', 'CVC': '6780', 'expire': '10/26'}
{'date': '2022/12/08 16:45:57', 'ID': '67e7c08e-770f-11ed-a6ad-309c23ff59e3', 'country': 'American Samoa', 'name': 'Jennifer Richardson', 'number': '2251500213338198', 'CVC': '530', 'expire': '09/26'}
{'date': '2022/12/08 16:45:57', 'ID': '680828a6-770f-11ed-a6ad-309c23ff59e3', 'country': 'Tokelau', 'name': 'Jennifer Barnett', 'number': '30220179050750', 'CVC': '553', 'expire': '06/30'}
{'date': '2022/12/08 16:45:57', 'ID': '6827e7fe-770f-11ed-a6ad-309c23ff59e3', 'country': 'Laos', 'name': 'Jared Ross', 'number': '4532338125190333', 'CVC': '180', 'expire': '12/29'}
{'date': '2022/12/08 16:45:57', 'ID': '6847cd58-770f-11ed-a6ad-309c23ff59e3', 'country': 'Congo', 'name': 'Deborah Wagner', 'number': '30382598805923', 'CVC': '826', 'expire': '01/27'}
{'date': '2022/12/08 16:45:57', 'ID': '68678fb2

{'date': '2022/12/08 16:46:04', 'ID': '6c0c7ca4-770f-11ed-a6ad-309c23ff59e3', 'country': 'Venezuela', 'name': 'Nathan Strickland', 'number': '60423143944', 'CVC': '197', 'expire': '04/27'}
{'date': '2022/12/08 16:46:04', 'ID': '6c1d895e-770f-11ed-a6ad-309c23ff59e3', 'country': 'Panama', 'name': 'Katherine Hernandez', 'number': '3506508779494108', 'CVC': '680', 'expire': '10/26'}
{'date': '2022/12/08 16:46:04', 'ID': '6c2f1714-770f-11ed-a6ad-309c23ff59e3', 'country': 'Cocos Islands', 'name': 'Scott Miller', 'number': '341386538377699', 'CVC': '150', 'expire': '07/25'}
{'date': '2022/12/08 16:46:04', 'ID': '6c40380a-770f-11ed-a6ad-309c23ff59e3', 'country': 'Czech Republic', 'name': 'Micheal Adams', 'number': '3555785945280302', 'CVC': '676', 'expire': '12/26'}
{'date': '2022/12/08 16:46:04', 'ID': '6c6136cc-770f-11ed-a6ad-309c23ff59e3', 'country': 'French S. Territories', 'name': 'Erika Parks', 'number': '4329519487097443016', 'CVC': '409', 'expire': '03/23'}
{'date': '2022/12/08 16:46:0

...

{'date': '2022/12/07 16:46:15', 'ID': '5fe8ddb6-7712-11ed-8bd8-309c23ff59e3', 'country': 'Iraq', 'name': 'Michael Simmons', 'number': '3557735215077928', 'CVC': '391', 'expire': '11/26'}
{'date': '2022/12/07 16:46:15', 'ID': '600acef8-7712-11ed-8bd8-309c23ff59e3', 'country': 'Guinea-bissau', 'name': 'Alex Freeman', 'number': '4343395260318', 'CVC': '737', 'expire': '12/30'}
{'date': '2022/12/07 16:46:15', 'ID': '601ca858-7712-11ed-8bd8-309c23ff59e3', 'country': 'East Timor', 'name': 'Ian Barker', 'number': '3590461916482338', 'CVC': '146', 'expire': '06/30'}
{'date': '2022/12/07 16:46:15', 'ID': '602e2d44-7712-11ed-8bd8-309c23ff59e3', 'country': 'Tunisia', 'name': 'Pamela Simpson', 'number': '4173324799497984945', 'CVC': '111', 'expire': '02/27'}
{'date': '2022/12/07 16:46:15', 'ID': '604efe98-7712-11ed-8bd8-309c23ff59e3', 'country': 'Tuvalu', 'name': 'Daniel Miller', 'number': '4697764488903', 'CVC': '540', 'expire': '07/29'}
{'date': '2022/12/07 16:46:15', 'ID': '60608d52-7712-11ed-8

{'date': '2022/12/07 16:46:15', 'ID': '6352af40-7712-11ed-8bd8-309c23ff59e3', 'country': 'Colombia', 'name': 'Robert Murphy', 'number': '4664248037609702', 'CVC': '755', 'expire': '02/26'}
{'date': '2022/12/07 16:46:15', 'ID': '6372a296-7712-11ed-8bd8-309c23ff59e3', 'country': 'Thailand', 'name': 'Maria Moore', 'number': '4331573485661902', 'CVC': '972', 'expire': '10/28'}
{'date': '2022/12/07 16:46:15', 'ID': '63937a0c-7712-11ed-8bd8-309c23ff59e3', 'country': 'Turkmenistan', 'name': 'Scott Williams', 'number': '4193723356775961', 'CVC': '883', 'expire': '05/28'}
{'date': '2022/12/07 16:46:15', 'ID': '63a4ac64-7712-11ed-8bd8-309c23ff59e3', 'country': 'Italy', 'name': 'Dr. Joshua Rivas DVM', 'number': '4580921164785344', 'CVC': '864', 'expire': '11/28'}
{'date': '2022/12/07 16:46:15', 'ID': '63b5e25e-7712-11ed-8bd8-309c23ff59e3', 'country': 'El Salvador', 'name': 'Kristin Sampson', 'number': '502046298902', 'CVC': '586', 'expire': '02/26'}
{'date': '2022/12/07 16:46:15', 'ID': '63c6e87e

{'date': '2022/12/07 16:46:15', 'ID': '6768669c-7712-11ed-8bd8-309c23ff59e3', 'country': 'Turkmenistan', 'name': 'Steven Vasquez', 'number': '30292529990589', 'CVC': '932', 'expire': '07/30'}
{'date': '2022/12/07 16:46:15', 'ID': '677980f8-7712-11ed-8bd8-309c23ff59e3', 'country': 'Panama', 'name': 'Claire Booker', 'number': '213163357508539', 'CVC': '178', 'expire': '01/23'}
{'date': '2022/12/07 16:46:15', 'ID': '678ad79a-7712-11ed-8bd8-309c23ff59e3', 'country': 'Eritrea', 'name': 'Matthew Watts', 'number': '38304051928101', 'CVC': '981', 'expire': '02/29'}
{'date': '2022/12/07 16:46:15', 'ID': '679c09ca-7712-11ed-8bd8-309c23ff59e3', 'country': 'Azerbaijan', 'name': 'Jacob Perez', 'number': '36628835594363', 'CVC': '2331', 'expire': '05/27'}
{'date': '2022/12/07 16:46:15', 'ID': '67bcfe64-7712-11ed-8bd8-309c23ff59e3', 'country': 'Reunion', 'name': 'David Santana', 'number': '4556356755442289605', 'CVC': '59', 'expire': '04/25'}
{'date': '2022/12/07 16:46:15', 'ID': '67dd8f62-7712-11ed-

{'date': '2022/12/07 16:46:15', 'ID': '6b362f3e-7712-11ed-8bd8-309c23ff59e3', 'country': 'Tuvalu', 'name': 'Scott Mills', 'number': '38407384296516', 'CVC': '725', 'expire': '05/29'}
{'date': '2022/12/07 16:46:15', 'ID': '6b57eef8-7712-11ed-8bd8-309c23ff59e3', 'country': 'New Zealand', 'name': 'Miss Sara Holloway', 'number': '2249482821946768', 'CVC': '510', 'expire': '10/25'}
{'date': '2022/12/07 16:46:15', 'ID': '6b6afb88-7712-11ed-8bd8-309c23ff59e3', 'country': 'Estonia', 'name': 'Miss Cynthia Ho', 'number': '213147269109338', 'CVC': '750', 'expire': '10/26'}
{'date': '2022/12/07 16:46:15', 'ID': '6b7cb4e0-7712-11ed-8bd8-309c23ff59e3', 'country': 'Israel', 'name': 'Dr. Elizabeth Underwood', 'number': '675976862770', 'CVC': '967', 'expire': '11/31'}
{'date': '2022/12/07 16:46:15', 'ID': '6b8ff7da-7712-11ed-8bd8-309c23ff59e3', 'country': 'Nepal', 'name': 'Tara Allen', 'number': '4172282900842061', 'CVC': '216', 'expire': '04/24'}
{'date': '2022/12/07 16:46:15', 'ID': '6ba20f10-7712-11

KeyboardInterrupt: 